In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore", category=Warning)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv("data_clean.csv")

In [3]:
data.head()

,type de location,accès au téléphone,pays_Kenya,pays_Rwanda,pays_Tanzania,pays_Uganda,Group Age_[10-20],Group Age_[20-30],Group Age_[30-40],Group Age_[40-50],...,relation avec le chef de famille_Other relative,relation avec le chef de famille_Parent,relation avec le chef de famille_Spouse,année_2016,année_2017,année_2018,taille du ménage,age,education_level_encoded,compte bancaire
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,3,24,3.0,Yes
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5,70,1.0,No
2,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,5,26,4.0,Yes
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5,34,2.0,No
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,8,26,2.0,No


#### Fonction : Entrainement & Evaluation du modèle 

In [4]:
def model(data, model_algo):
    # Séparation des features de la Target     
    X = data.drop("compte bancaire", axis="columns")
    Y = data["compte bancaire"]
    # Division des données en ensemble d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    # Entraînement du modèle de régression logistique
    model = model_algo
    model.fit(X_train, y_train)
    # Prédiction sur l'ensemble de test
    y_pred = model.predict(X_test)
    # Évaluation de la performance du modèle
    accuracy = accuracy_score(y_test, y_pred)
    # print("Précision du modèle :", accuracy)
    print(f"===================   {model_algo}   ====================")
    print()
    print(classification_report(y_test, y_pred))
    return accuracy

#### Fonction : Equilibrage des données 

In [5]:
def balance_dataset(df, nb):
    grouped = df.groupby('compte bancaire')
    df_sampled = grouped.apply(lambda x: x.sample(nb) if len(x) > nb else x)
    df_sampled = df_sampled.reset_index(drop=True)
    return df_sampled

#### Fonction : Comparation des performances des modèles

In [6]:
def resultat_compare(data, model_algo: list, balance=False, nb=0):
    data_dict = []
    if balance:
        data = balance_dataset(data, nb)   
    for i in model_algo:
        if i == DecisionTreeClassifier():
            data['compte bancaire'] = label_encoder.fit_transform(data['compte bancaire'])
        accuracy = model(data, i)
        dictionnaire = {
            "Modèle": i,
            "Précision": accuracy,
        }
        data_dict.append(dictionnaire) 
    data_dict = pd.DataFrame(data_dict)
    return data_dict

In [7]:
Algo = [LogisticRegression(), DecisionTreeClassifier(), DummyClassifier(), GaussianNB()]

#### Affichage des résultats

In [9]:
resultat_compare(data, Algo, balance=False, nb=1000)

===================   LogisticRegression()   ====================

              precision    recall  f1-score   support

          No       0.90      0.97      0.93      4005
         Yes       0.70      0.36      0.47       700

    accuracy                           0.88      4705
   macro avg       0.80      0.67      0.70      4705
weighted avg       0.87      0.88      0.87      4705

===================   DecisionTreeClassifier()   ====================

              precision    recall  f1-score   support

          No       0.90      0.90      0.90      4005
         Yes       0.41      0.40      0.40       700

    accuracy                           0.82      4705
   macro avg       0.65      0.65      0.65      4705
weighted avg       0.82      0.82      0.82      4705

===================   DummyClassifier()   ====================

              precision    recall  f1-score   support

          No       0.85      1.00      0.92      4005
         Yes       0.00      0.00  

,Modèle,Précision
0,LogisticRegression(),0.881828
1,DecisionTreeClassifier(),0.823379
2,DummyClassifier(),0.851222
3,GaussianNB(),0.804251
